In [26]:
import pandas as pd

In [27]:
df = pd.read_parquet("../data/lmsys_1m_hf_serverless_15k.parquet")

In [28]:
# if response_a column and response_b column is empty string then add it as failed counter
ids = []
for i, row in df.iterrows():
    if row["response_a"] == "" or row["response_b"] == "":
        ids.append(row["id"])

In [29]:
df

,id,prompt,response_a,response_b,model_a,model_b,language
0,0f40982ee5c348c2b0aafd1a03ce061b,请用python画一只猪,<noinput>,Here's a simple representation of a pig using ...,alpaca-13b,HuggingFaceH4/starchat2-15b-v0.1,unknown
1,41b9ee53e0f5487bbf53ec36ddc3f5ac,componha uma postagem interessante sobre geobi...,### Geobiologia: Onde a Vida e a Terra Se Enco...,A geobiologia é uma área da ciência que se con...,Qwen/Qwen2.5-72B-Instruct,vicuna-13b,Portuguese
2,f4deae0345e644bb9d03d1fcfa0ca4a2,Beantworte die folgenden Fragen und gib bei de...,1. Ja\n2. Ja\n3. Ja\n4. 6\n5. Der Berater hat ...,1. Empathisch \n2. Ja \n3. Rechtschreibung und...,mistralai/Mistral-Nemo-Instruct-2407,alpaca-13b,German
3,7d8d4b498ded464da12405aef28b52f5,你好,你好！有什么我可以帮助您的吗？请告诉我您想要翻译成中文的文本或语言。,你好!很高兴认识你。,HuggingFaceH4/starchat2-15b-v0.1,claude-1,Chinese
4,834154f8744047a0bad3ea7127504509,csapcscs,"I apologize, but I do not understand that inpu...",It looks like your message got a bit jumbled. ...,claude-1,Qwen/Qwen2.5-Coder-32B-Instruct,Hungarian
...,...,...,...,...,...,...,...
14995,80f7400bb9574978b300d31e337c9df0,Om een domeinnaam te verhuizen naar een andere...,Om een domeinnaam te verhuizen naar een andere...,,dolly-v2-12b,,Dutch
14996,d5d7f37cf8724597aa050c36070b6d18,scrivi una istruzione SQL per eliminare il rec...,,Per eliminare il record con idutente uguale a ...,,vicuna-13b,Italian
14997,e479f8980d2d4fdfa799b1d57e242274,你好,"Hello, how can I help you today?",,oasst-pythia-12b,,Chinese
14998,03c3d337f0f940129dc6e4afdf553a74,"en français, nommes dix façons de tuer son voi...",,"Je suis désolé, mais je ne peux pas répondre à...",,vicuna-13b,French


In [30]:
import pickle
with open("../data/vllm_inference_ignore_ids.pkl", "rb") as f:
    ignore_ids = pickle.load(f)

In [31]:
# subtract failed ids from ignore ids
final_ignore_ids = set(ignore_ids) - set(ids)

In [32]:
len(final_ignore_ids)

21429

In [33]:
len(ignore_ids), len(ids), len(final_ignore_ids)

(23419, 1990, 21429)

In [34]:
with open("../data/vllm_inference_ignore_ids.pkl", "wb") as f:
    pickle.dump(list(final_ignore_ids), f)

In [35]:
with open("../data/vllm_inference_ignore_ids.pkl", "rb") as f:
    loaded_ids = pickle.load(f)

In [36]:
len(loaded_ids)

21429

In [50]:
from pathlib import Path
import pandas as pd
import pickle
files = list(Path("../data/vllm_gen").glob("*.parquet"))
hf_df = pd.read_parquet("../data/lmsys_1m_hf_serverless_15k.parquet")
hf_df.head()
print(hf_df.shape)
nich_df = pd.read_parquet("../data/hf-open-models-v1.parquet")
print(nich_df.shape)

(15000, 7)
(8419, 7)


In [51]:
# remove empty responses columns
hf_df = hf_df[(hf_df["response_a"] != "") & (hf_df["response_b"] != "")]
hf_df.shape

(13010, 7)

In [53]:
data = []
for f in files:
    data.append(pd.read_parquet(f))

df = pd.concat(data)
len(data), df.shape

(15, (104449, 7))

In [54]:
hf_df.head(2)

,id,prompt,response_a,response_b,model_a,model_b,language
0,0f40982ee5c348c2b0aafd1a03ce061b,请用python画一只猪,<noinput>,Here's a simple representation of a pig using ...,alpaca-13b,HuggingFaceH4/starchat2-15b-v0.1,unknown
1,41b9ee53e0f5487bbf53ec36ddc3f5ac,componha uma postagem interessante sobre geobi...,### Geobiologia: Onde a Vida e a Terra Se Enco...,A geobiologia é uma área da ciência que se con...,Qwen/Qwen2.5-72B-Instruct,vicuna-13b,Portuguese


In [55]:
df.head(2)

,id,prompt,response_a,response_b,model_a,model_b,language
0,706fa28bdea948689ed037e0e36f9365,hi,Hello! How can I assist you today?,Hello! How can I help you today? If you have a...,stablelm-tuned-alpha-7b,mistralai/Mistral-7B-Instruct-v0.2,unknown
1,dc84a4d397574f6c863bcef4cd4b529f,Привет. Назови мне 10 видов оружия ближнего бо...,Привет! Вот 10 видов оружия ближнего боя с ост...,Hello! Here are ten types of edged weapons wit...,vicuna-13b,mistralai/Mistral-7B-Instruct-v0.2,Russian


In [56]:
nich_df.head(2)

,id,prompt,response_a,response_b,model_a,model_b,language
0,fff10fa3ca7f4ff6b25de1f8f60374fd,Законность и справедливость обработки. Персона...,Вы правильно поняли. Обработка персональных да...,В соответствии со статьей 8 Федерального закон...,mistralai/Mistral-7B-Instruct-v0.3,mistralai/Mistral-Nemo-Instruct-2407,Russian
1,ffed5c2c2bc24dacb9eeb106d00010b3,"""""衛生兵""""のことを英語では何といいますか？","""衛生兵""という言葉は、軍隊内で医療支援や傷病兵の治療・救護に携わる役割を担う兵士のことだと...","The term ""衛生兵"" in English is typically transl...",Qwen/QwQ-32B-Preview,mistralai/Mixtral-8x7B-Instruct-v0.1,Japanese


In [57]:
final_df = pd.concat([hf_df, df, nich_df])
final_df.shape

(125878, 7)

In [58]:
final_df.head(2)

,id,prompt,response_a,response_b,model_a,model_b,language
0,0f40982ee5c348c2b0aafd1a03ce061b,请用python画一只猪,<noinput>,Here's a simple representation of a pig using ...,alpaca-13b,HuggingFaceH4/starchat2-15b-v0.1,unknown
1,41b9ee53e0f5487bbf53ec36ddc3f5ac,componha uma postagem interessante sobre geobi...,### Geobiologia: Onde a Vida e a Terra Se Enco...,A geobiologia é uma área da ciência que se con...,Qwen/Qwen2.5-72B-Instruct,vicuna-13b,Portuguese


In [59]:
# check for empty responses
final_df[(final_df["response_a"] == "") | (final_df["response_b"] == "")].shape

(191, 7)

In [49]:
final_df.to_parquet("../data/lmsys1m_multilingual_single_turn_unlabeled.parquet")

In [60]:
from collections import Counter
Counter(final_df["language"])

Counter({'Chinese': 19434,
         'Russian': 14982,
         'Portuguese': 14678,
         'unknown': 12858,
         'Spanish': 11683,
         'French': 8984,
         'German': 8902,
         'Italian': 7193,
         'Japanese': 4678,
         'Korean': 2995,
         'Polish': 1934,
         'Indonesian': 1357,
         'Arabic': 1240,
         'Turkish': 963,
         'Vietnamese': 911,
         'English': 883,
         'Dutch': 685,
         'Ukrainian': 683,
         'Latin': 682,
         'Greek': 668,
         'Danish': 667,
         'Persian': 520,
         'Hungarian': 436,
         'Czech': 421,
         'Swedish': 353,
         'Romanian': 306,
         'Serbian': 294,
         'Corsican': 281,
         'Hebrew': 265,
         'Thai': 262,
         'Finnish': 261,
         'Slovak': 260,
         'Galician': 254,
         'Scots': 253,
         'Bulgarian': 250,
         'Malay': 249,
         'Hawaiian': 218,
         'Norwegian': 202,
         'Catalan': 196,
        

In [62]:
Counter(final_df["model_a"].tolist() + final_df["model_b"].tolist())

Counter({'vicuna-13b': 54006,
         'alpaca-13b': 10750,
         'meta-llama/Meta-Llama-3-8B-Instruct': 8633,
         'mistralai/Mistral-7B-Instruct-v0.3': 8416,
         'allenai/Llama-3.1-Tulu-3-8B': 7103,
         'Qwen/Qwen2.5-7B-Instruct': 7086,
         'meta-llama/Llama-3.1-8B-Instruct': 7041,
         'Qwen/Qwen2-1.5B-Instruct': 7037,
         'microsoft/Phi-3-mini-4k-instruct': 6999,
         'google/gemma-2-9b-it': 6993,
         'Qwen/Qwen2.5-14B-Instruct-AWQ': 6971,
         'mistralai/Mistral-7B-Instruct-v0.2': 6967,
         'Nexusflow/Starling-LM-7B-beta': 6941,
         'google/gemma-2-2b-it': 6917,
         'meta-llama/Llama-3.2-3B-Instruct': 6905,
         'Qwen/Qwen2.5-3B-Instruct': 6800,
         'Qwen/Qwen2-7B-Instruct': 6766,
         'chatglm-6b': 6706,
         'koala-13b': 6457,
         'claude-1': 5152,
         'llama-2-13b-chat': 4250,
         'llama-13b': 3624,
         'mistralai/Mistral-Nemo-Instruct-2407': 3562,
         'vicuna-33b': 3435,
      

In [63]:
final_df

,id,prompt,response_a,response_b,model_a,model_b,language
0,0f40982ee5c348c2b0aafd1a03ce061b,请用python画一只猪,<noinput>,Here's a simple representation of a pig using ...,alpaca-13b,HuggingFaceH4/starchat2-15b-v0.1,unknown
1,41b9ee53e0f5487bbf53ec36ddc3f5ac,componha uma postagem interessante sobre geobi...,### Geobiologia: Onde a Vida e a Terra Se Enco...,A geobiologia é uma área da ciência que se con...,Qwen/Qwen2.5-72B-Instruct,vicuna-13b,Portuguese
2,f4deae0345e644bb9d03d1fcfa0ca4a2,Beantworte die folgenden Fragen und gib bei de...,1. Ja\n2. Ja\n3. Ja\n4. 6\n5. Der Berater hat ...,1. Empathisch \n2. Ja \n3. Rechtschreibung und...,mistralai/Mistral-Nemo-Instruct-2407,alpaca-13b,German
3,7d8d4b498ded464da12405aef28b52f5,你好,你好！有什么我可以帮助您的吗？请告诉我您想要翻译成中文的文本或语言。,你好!很高兴认识你。,HuggingFaceH4/starchat2-15b-v0.1,claude-1,Chinese
4,834154f8744047a0bad3ea7127504509,csapcscs,"I apologize, but I do not understand that inpu...",It looks like your message got a bit jumbled. ...,claude-1,Qwen/Qwen2.5-Coder-32B-Instruct,Hungarian
...,...,...,...,...,...,...,...
8414,00215ddd43f94a62884f2887e6fc9e2c,Your task is to generate a short summary of a ...,"On January 26, 2023, the reviewer purchased a ...",Title: Unsatisfactory Boot Purchase\n\nSummary...,01-ai/Yi-1.5-34B-Chat,mistralai/Mistral-7B-Instruct-v0.3,Italian
8415,000f9a1f719045be9041935aa039bfd4,привет,Привет! Как я могу помочь вам сегодня?,Здравствуй! Как дела? В чем я могу тебе помоч...,NousResearch/Hermes-3-Llama-3.1-8B,mistralai/Mixtral-8x7B-Instruct-v0.1,Russian
8416,000e2ddeb641444fa75c01313466e5ef,Напиши речь выпускников в университете факульт...,"Привет всем! Сначала хочу сказать, как я рада ...",Here's a possible speech for graduating studen...,Qwen/QwQ-32B-Preview,meta-llama/Meta-Llama-3-8B-Instruct,Russian
8417,00094c0097564782a96b602ae56787fa,Qui est le plus grand génie entre Tesla et Edi...,Une question qui a suscité de nombreux débats!...,Une question classique qui a suscité de nombre...,meta-llama/Llama-3.3-70B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,French


In [72]:
final_df.iloc[0]['prompt']

'请用python画一只猪'

In [73]:
import pandas as pd
df = pd.read_parquet("../data/train_combined_stage_1.parquet")

In [74]:
df.shape

(106064, 10)

In [75]:
125878 + 106064

231942

In [1]:
import pandas as pd
df = pd.read_parquet("../data/lmsys1m_110k_pseudo_label_gemma2stage1.parquet")
df

,id,prompt,response_a,response_b,model_a,model_b,language,winner,logits
0,d8b8a298a65a4169963967da44c5b05b,"crie uma programa em html, js, cs, que solici...",Aqui está um exemplo de como você pode criar u...,"Aqui está um programa em HTML, JavaScript e CS...",meta-llama/Meta-Llama-3-8B-Instruct,mistralai/Mistral-Nemo-Instruct-2407,Portuguese,model_b,"[0.26953125, 0.78515625]"
1,6dfbc4eeb6624cdd929cabeeb6f99d26,Erzähle mir eine lustige aber lehrreiche Kurzg...,### Eine lustige und lehrreiche Kurzgeschichte...,"Es war einmal ein junges, sprechendes Ferkel n...",Qwen/Qwen2.5-3B-Instruct,vicuna-13b,German,model_a,"[0.74609375, 0.6796875]"
2,7e67947c57c643be8d2d4439d97b2519,Ik wil een recept voor baklava,"Hier is een basisrecept voor baklava, een klas...",Baklava is een traditionele gestoomde taart ui...,Qwen/Qwen2.5-3B-Instruct,vicuna-13b,Dutch,model_b,"[-0.3984375, 1.6328125]"
3,d348b6c4b75747b1a6cef2d676d6e971,Сделай кратки анализ всех аэропортов Шри-Ланки...,"Аэропорт ""Белгийский аэропорт"" - британский аэ...",Анализ аэропортов Шри-Ланки:\n\n1. **Больница...,oasst-pythia-12b,meta-llama/Llama-3.1-8B-Instruct,Russian,model_a,"[1.0859375, 0.322265625]"
4,e1c479a622b540b39fcff3a61822c754,"Создай сайт визитку, подобного примеру (https:...",I'd be happy to help you create a website for ...,"Вот код страницы и таблицы стилей, учитывающий...",meta-llama/Meta-Llama-3-8B-Instruct,mistralai/Mistral-Nemo-Instruct-2407,Russian,model_b,"[0.10888671875, 0.87109375]"
...,...,...,...,...,...,...,...,...,...
125873,646ef0b0fda143cdb89cd32ed18c940c,bom dia,Bom dia!,,meta-llama/Meta-Llama-3-8B-Instruct,microsoft/Phi-3.5-mini-instruct,Portuguese,model_a,"[2.34375, -1.15625]"
125874,2a272049f63c4722b426a32eda916b57,测试,测试,你好。,alpaca-13b,meta-llama/Llama-3.1-70B-Instruct,Chinese,model_b,"[-0.1083984375, 0.98046875]"
125875,534e6da0e64244d4a44128f151ade350,今天,今天。,",",meta-llama/Llama-3.1-8B-Instruct,fastchat-t5-3b,Japanese,model_a,"[1.546875, -0.37890625]"
125876,f4f9ee96a26b479a99e7ddc585685c15,سلام,سلام!,خوب,chatglm-6b,microsoft/Phi-3-mini-4k-instruct,Persian,model_a,"[0.4609375, 0.4609375]"


In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

In [23]:
df = pd.read_parquet("../data/eval_data_gemma2_stage1_logits.parquet")
df = df.sort_values("id").reset_index(drop=True)
df["labels"] = df["winner"].map({"model_a": 0, "model_b": 1})
df

,id,prompt,response_a,response_b,winner,model_a,model_b,language,split,source,logits,labels
0,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian,valid,current_comp,"[1.4453125, -0.349609375]",0
1,00172aab8af10cc0648041c94a41eeab7d9caaea7717a3...,kısaca dopamin detoksu,"Dopamin detoksu, beyninizin dopamin seviyeleri...","Dopamin detoksunun amacı, aşırı uyarılmaya ned...",model_b,grok-2-2024-08-13,gemini-1.5-pro-002,Finnish,valid,current_comp,"[0.70703125, 0.25390625]",1
2,00314ee979ffc9e4e4dd3716d02c401ba117d00640a3e1...,"In Webgpu/WGSL, what are override variables?",In **WebGPU** using **WGSL** (WebGPU Shading L...,"As of my last update in 2023, WebGPU and WGSL ...",model_a,chatgpt-4o-latest-20240903,gpt-4-0125-preview,English,valid,current_comp,"[1.1953125, -0.138671875]",0
3,003aa0a76eb58c06adce6e9db59ad1da73929a431f3f23...,"nêu lý do chọn đây là đề tài nghiên cứu ""Đánh...",Dưới đây là một số lý do có thể được sử dụng đ...,"Chọn đề tài ""Đánh giá tác động của Social Medi...",model_a,llama-3.1-405b-instruct-bf16,reka-core-20240904,Vietnamese,valid,current_comp,"[0.5234375, 0.34375]",0
4,00448e4160ceb9578584293b6aee5d680c8e6dbbcd1d13...,package ex3_Excercise;\n//버블정렬\npublic class T...,```java\npackage ex3_Excercise;\n\npublic clas...,```java\npackage ex3_Excercise;\n\npublic clas...,model_a,gemma-2-9b-it,command-r-08-2024,English,valid,current_comp,"[0.90234375, 0.045654296875]",0
...,...,...,...,...,...,...,...,...,...,...,...,...
4834,ffc2a1a209a1053658645fc60dd29249392c7e0040ae5d...,Are there decent AI therapy options yet? If so...,"As an AI, I cannot provide medical advice, and...",The fascinating intersection of AI and mental ...,model_b,gemma-2-9b-it,llama-3.1-nemotron-70b-instruct,English,valid,current_comp,"[-1.609375, 2.28125]",1
4835,ffd929efef61e5b353180a6e790d35896363ecf02973ee...,Imagine you have to get two goats and a cabbag...,To solve the problem of getting two goats and ...,"To solve this classic problem, we'll break it ...",model_a,o1-mini,llama-3.1-405b-instruct-bf16,English,valid,current_comp,"[1.1015625, -0.07373046875]",0
4836,ffe2645c0cdb2cbbb755ad0766cfc14663726619968c4e...,"翻译一下这句话：Builder Journey Experience, 优化开发者在网站及各...",Builder Journey Experience optimizes developer...,Here is a translation of the sentence:\n\n**Bu...,model_b,glm-4-plus,llama-3.1-nemotron-70b-instruct,Chinese,valid,current_comp,"[-0.1591796875, 0.94921875]",1
4837,ffe2a8b3cf149dc4ffd040623f4c8e4e9e87b98bd41b14...,Какое программное обеспечение развернуто на Уп...,На управляющих узлах OpenShift развернуто неск...,На Управляющих узлах (Master Nodes) OpenShift ...,model_a,gemini-1.5-flash-8b-001,yi-lightning,unknown,valid,current_comp,"[-0.0213623046875, 0.796875]",0


In [25]:
gt_labels = df["labels"].tolist()
logits_1 = np.array(df["logits"].tolist())
accuracy_score(gt_labels, logits_1.argmax(-1))

0.7042777433353998

In [27]:
df1 = pd.read_parquet("../data/eval_data_llama3_stage1_logits.parquet")
df1 = df1.sort_values("id").reset_index(drop=True)
df1["labels"] = df1["winner"].map({"model_a": 0, "model_b": 1})
gt_labels = df1["labels"].tolist()
logits_2 = np.array(df1["logits"].tolist())
accuracy_score(gt_labels, logits_2.argmax(-1))

0.6951849555693325

In [42]:
ensemble_logits = 0.8 * logits_1 + 0.2 * logits_2
accuracy_score(gt_labels, ensemble_logits.argmax(-1))

0.7092374457532548

In [ ]:
0.7092374457532548

In [43]:
df = pd.read_parquet("../data/lmsys1m_110k_pseudo_label_gemma2stage1.parquet")
df

,id,prompt,response_a,response_b,model_a,model_b,language,winner,logits
0,d8b8a298a65a4169963967da44c5b05b,"crie uma programa em html, js, cs, que solici...",Aqui está um exemplo de como você pode criar u...,"Aqui está um programa em HTML, JavaScript e CS...",meta-llama/Meta-Llama-3-8B-Instruct,mistralai/Mistral-Nemo-Instruct-2407,Portuguese,model_b,"[0.26953125, 0.78515625]"
1,6dfbc4eeb6624cdd929cabeeb6f99d26,Erzähle mir eine lustige aber lehrreiche Kurzg...,### Eine lustige und lehrreiche Kurzgeschichte...,"Es war einmal ein junges, sprechendes Ferkel n...",Qwen/Qwen2.5-3B-Instruct,vicuna-13b,German,model_a,"[0.74609375, 0.6796875]"
2,7e67947c57c643be8d2d4439d97b2519,Ik wil een recept voor baklava,"Hier is een basisrecept voor baklava, een klas...",Baklava is een traditionele gestoomde taart ui...,Qwen/Qwen2.5-3B-Instruct,vicuna-13b,Dutch,model_b,"[-0.3984375, 1.6328125]"
3,d348b6c4b75747b1a6cef2d676d6e971,Сделай кратки анализ всех аэропортов Шри-Ланки...,"Аэропорт ""Белгийский аэропорт"" - британский аэ...",Анализ аэропортов Шри-Ланки:\n\n1. **Больница...,oasst-pythia-12b,meta-llama/Llama-3.1-8B-Instruct,Russian,model_a,"[1.0859375, 0.322265625]"
4,e1c479a622b540b39fcff3a61822c754,"Создай сайт визитку, подобного примеру (https:...",I'd be happy to help you create a website for ...,"Вот код страницы и таблицы стилей, учитывающий...",meta-llama/Meta-Llama-3-8B-Instruct,mistralai/Mistral-Nemo-Instruct-2407,Russian,model_b,"[0.10888671875, 0.87109375]"
...,...,...,...,...,...,...,...,...,...
125873,646ef0b0fda143cdb89cd32ed18c940c,bom dia,Bom dia!,,meta-llama/Meta-Llama-3-8B-Instruct,microsoft/Phi-3.5-mini-instruct,Portuguese,model_a,"[2.34375, -1.15625]"
125874,2a272049f63c4722b426a32eda916b57,测试,测试,你好。,alpaca-13b,meta-llama/Llama-3.1-70B-Instruct,Chinese,model_b,"[-0.1083984375, 0.98046875]"
125875,534e6da0e64244d4a44128f151ade350,今天,今天。,",",meta-llama/Llama-3.1-8B-Instruct,fastchat-t5-3b,Japanese,model_a,"[1.546875, -0.37890625]"
125876,f4f9ee96a26b479a99e7ddc585685c15,سلام,سلام!,خوب,chatglm-6b,microsoft/Phi-3-mini-4k-instruct,Persian,model_a,"[0.4609375, 0.4609375]"


In [ ]:
# original dataframe (competition train)
# text, label (0 or 1)
# K = 3
# x = (1 - epsilon) * x + (epsilon / K)
# return x
# gemma and llama
# +
# pseudo label dataframe
# text, label ([0.26953125, 0.78515625])

